## Content-based Recommendation Systems

#### Bộ cơ sở dữ liệu gồm 100,000 rating từ  943 user cho 1682 bộ phim:
+ với mỗi user rate ít nhất 20 bộ phim, nên có những bộ phim mà user đó chưa rate và việc chúng ta cần làm là lấp đầy ma trận rating(lấp đầy các bộ phim mà user đó chưa rate)
. 
+ một bộ phim có thể thuộc nhiều thể loại(trong tất cả 19 thể loại phim) , đặc tính này được thể hiện bằng một vector hàng multi-hot coding trong file u.time, tạm thời kí hiệu đặc tính này là X

+ các thể loại phim gồm : unknown, Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western.
#### Các bước cần làm để xây dựng model cho từng user: 
+ đối với mỗi User lấy ra danh sách các phim mà user đó đã rate và giá trị rate 
+ sau khi lấy ra , ứng với mỗi bộ phim lấy ra đặc tính X của phim đó (vector hàng [1,19])
+ hàm mất mát cho mô hình của mỗi user (Ridge Regression):
$$ \Gamma_n = \frac{1}{2s_n}\parallel \widehat{X}_nW_n+b_ne_n-\widehat{y}_n \parallel_2^2+\frac{\lambda}{2s_n}\parallel w_n \parallel_2^2 $$
* $s_n$ là số lượng các item user đã rate
* $ \widehat{y}_n $ : trích các thành phần được rate từ cột thứ n(số sao rate cho mỗi phim)
* $ \widehat{X}_n $ : là sub matrix của ma trận vec tơ X , được tạo bằng cách trích các hàng tương ứng với các items đã được rate bởi user n
* $e_n$ : là ma trận vector cột chứa $s_n$ phần tử bằng 1 
##### Dùng BGD hoặc SGD để tính tìm $w_n$ và $b_n$ từ đó tính được lượng rate cho các phim mà user đó chưa rate





### Hiện thực phương pháp trên

In [18]:
import pandas as pd
# reading user file
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [17]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
user_id       943 non-null int64
age           943 non-null int64
sex           943 non-null object
occupation    943 non-null object
zip_code      943 non-null object
dtypes: int64(2), object(3)
memory usage: 36.9+ KB


#### Đọc bảng dữ liệu Y (lượng sao user rate cho các phim)

In [27]:
#reading ratings files:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

# convert to array
rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

print(ratings_base.shape)
ratings_base.head()
#ratings_base.info()
#rate_train = ratings_base.as_matrix()
#rate_test = ratings_test.as_matrix()

(90570, 4)


c:\users\ladyboy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\users\ladyboy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


#### Load vector X miêu tả đặc tính của  1682 bộ phim thuộc trong 19 thể loại (1 bộ có thể thuộc nhiều thể loại)

In [24]:
#load items
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
X0 = items.as_matrix()
# lấy 19 phần tử cuối cùng
X_train_counts = X0[:, -19:]
print(X_train_counts.shape)
X_train_counts

(1682, 19)


c:\users\ladyboy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

#### dùng sklearn.feature_extraction để trích xuất đặc tính 1 lần nữa (để giảm tính toán cho models)

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

In [32]:
# đây chính là vector X sau khi được trích xuất một lần nữa
print(X_train_counts[1])
print(tfidf[1])
# so sánh trước và sau khi trích xuất bằng tfidf

[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0.   0.54 0.65 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.54 0.   0.  ]


#### Với mỗi user tìm những bộ phim mà user đó đã rate và rating ứng với từng bộ phim đó

In [33]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores), time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # cộng thêm 1 cho user_id khi kiểm tra điều kiêm trong bẳng rate_matrix 
    # vì trong rate_matrix id bắt đầu từ 1 còn trong python bắt đầu từ 0
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0 
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

#### Tìm model cho từng user

In [9]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

n_users = users.shape[0]

d = tfidf.shape[1] # data dimension = 19(19 thể loại phim khác nhau)
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):    
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]
    
    clf.fit(Xhat, scores) 
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_


In [36]:
# predicted scores
Yhat = tfidf.dot(W) + b
# Yhat chính là matrix rating đã hoàn thiện khi dùng model đánh giá
print(Yhat.shape)
Yhat

(1682, 943)


array([[2.92, 4.  , 1.9 , ..., 5.04, 4.78, 3.17],
       [2.8 , 3.44, 3.3 , ..., 1.1 , 4.76, 3.7 ],
       [3.48, 1.56, 1.27, ..., 7.36, 4.19, 3.81],
       ...,
       [4.13, 4.1 , 3.02, ..., 9.54, 4.64, 3.27],
       [3.57, 3.45, 2.93, ..., 3.68, 3.66, 2.76],
       [4.13, 3.8 , 3.1 , ..., 9.54, 4.42, 3.93]])

In [11]:
# test với id thứ 11
n = 10
np.set_printoptions(precision=2) # 2 digits after . 
ids, scores = get_items_rated_by_user(rate_test, n)
Yhat[n, ids]
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [ 37 109 110 226 424 557 722 724 731 739]
True ratings     : [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]


#### Đánh giá sai số của toàn bộ mô hình

In [15]:
import math
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return math.sqrt(se/cnt)

print ('RMSE for training:', evaluate(Yhat, rate_train, W, b))
print ('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.908980456282672
RMSE for test    : 1.2703282700393037


Vậy sai số trung bình cho tập training là 0.9 sao , cho tập test là 1.27 sao